<a href="https://colab.research.google.com/github/Bertha-ding/20MA573-yuning-ding/blob/master/hw/hw7-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Example1 Consider CEV:
dSt = 0.03Stdt + σSγt dWt, S0 = 100.

In [20]:
import numpy as np
import scipy.stats as ss

class VanillaOption:
    def __init__(
        self,
        otype = 1, # 1: 'call'
                  # -1: 'put'
        strike = 110.,
        maturity = 1.,
        market_price = 10.,
        N = 1000.,
        n = 1000.,
        r=0.0475):
      self.otype = otype
      self.strike = strike
      self.maturity = maturity
      self.market_price = market_price #this will be used for calibration
      self.step = N
      self.path = n
      self.interest=r
      self.strike = strike
      self.maturity = maturity
      

    def payoff(self, s): #s: excercise price
      otype = self.otype
      k = self.strike
      maturity = self.maturity
      return max([0, (s - k)*otype])
    
    def EULER1D(self,S0):
      r = self.interest
      T = self.maturity
      N = self.step
      n = self.path
      delta = T/N
      Sdelta = []
      Sdelta.append(S0)
      t = np.linspace(0,T,N+1)
      s = 0
      
      for i in range(N):
        z = np.random.normal()
        Sdelta.append(Sdelta[i] + 0.003*Sdelta[i] + 0.2*Sdelta[i]*np.sqrt(delta)*z)
        p = payoff(self, Sdelta[i])
        s += p
      return np.exp(-r*T)*s/n
VanillaOption(strike=97)

In [10]:
class Gbm:
    def __init__(self,
                 init_state = 100.,
                 drift_ratio = .0475,
                 vol_ratio = .2
                ):
        self.init_state = init_state
        self.drift_ratio = drift_ratio
        self.vol_ratio = vol_ratio

def bsm_price(self, vanilla_option):
    s0 = self.init_state
    sigma = self.vol_ratio
    r = self.drift_ratio
    
    otype = vanilla_option.otype
    k = vanilla_option.strike
    maturity = vanilla_option.maturity
    
    d1 = (np.log(s0 / k) + (r + 0.5 * sigma ** 2) 
          * maturity) / (sigma * np.sqrt(maturity))
    d2 = d1 - sigma * np.sqrt(maturity)
    
    return (otype * s0 * ss.norm.cdf(otype * d1) #line break needs parenthesis
            - otype * np.exp(-r * maturity) * k * ss.norm.cdf(otype * d2))

Gbm.bsm_price = bsm_price    
gbm1 = Gbm()
option1 = VanillaOption()
print('>>>>>>>>>>call value is ' + str(gbm1.bsm_price(option1)))

>>>>>>>>>>call value is 11.989673615890688


Download option data from here and calibrate volatility and elasticity constant from 5 month
call options.

In [21]:
%cd~

!git clone https://github.com/Bertha-ding/20MA573-yuning-ding.git 
pass
%cd 20MA573-yuning-ding/src/
%ls

/root
Cloning into '20MA573-yuning-ding'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 222 (delta 27), reused 0 (delta 0), pack-reused 169
Receiving objects: 100% (222/222), 10.14 MiB | 43.10 MiB/s, done.
Resolving deltas: 100% (116/116), done.
/root/20MA573-yuning-ding/src
20optiondata2.dat  hw1_random_walk.ipynb  prj02.ipynb  Untitled0.ipynb
bsm.py             prj01.ipynb            prj03.ipynb


In [22]:
np_option_data1 = np.loadtxt('20optiondata2.dat', comments='#', delimiter=',')

print('>>>otype,>>>>>>> maturity, >>strike, >>>>>>option_price')
print(np_option_data1)

>>>otype,>>>>>>> maturity, >>strike, >>>>>>option_price
[[  1.           0.16666667  97.           5.32705461]
 [  1.           0.16666667  99.           3.86224255]
 [  1.           0.16666667 101.           2.7204371 ]
 [  1.           0.16666667 103.           2.1202793 ]
 [  1.           0.41666667  97.           7.23756307]
 [  1.           0.41666667  99.           5.95053461]
 [  1.           0.41666667 101.           5.2640122 ]
 [  1.           0.41666667 103.           4.97493422]]


In [0]:
import scipy.optimize as so
from bsm import *

gbm1 = Gbm(
    init_state = 100., 
    drift_ratio = .03,
    vol_ratio = .2)

'''================
define an error function
===================='''
def error_function(vol, gbm, option):
  gbm.vol_ratio = vol
  err = 0
  for i in range(len(option)):
    err = (err + abs((option_list[i]).market_price - gbm.bsm_price(option_list[i]))**2)  
  return err

  
'''==========
define a method to seek for an calibrate volatility
============'''

def implied_volatility(gbm, option_list):
  init_vol = .1 #initial guess
  return so.fmin(error_function, init_vol, args = (gbm, option_list), disp = 0)[0]


In [43]:
filter1 = np_option_data1[np_option_data1[:,1] == 5/12]
num_row = filter1.shape[0]
option_list = []

for i in range(num_row):
  option1 = VanillaOption(
      otype = filter1[i,0],
      strike = filter1[i,2],
      maturity = filter1[i,1],
      market_price = filter1[i,3]
  )   
  option_list.append(option1)
  calibrated_volatility = implied_volatility(gbm1, option_list)
print('The calibrated volatility for 5-mon is' ,calibrated_volatility)

The calibrated volatility for 5-mon is 0.20109375000000035
